In [1]:
import os
os.environ['JAX_PLATFORMS'] = "cpu"

import numpy as np
import jax.numpy as jnp
import jax

In [2]:
from vit import transformer

In [3]:
from vit import test_equality

In [4]:
params = test_equality._load_params("/home/dibya/nfs/siglip2_b16_224.npz")

In [5]:
model = transformer.ViT.create_from_pretrained(params, transformer.ViTConfig.B16())

In [6]:
from PIL import Image
image = np.asarray(Image.open('cat.jpg').resize((224, 224)))

In [7]:
pred_outputs = model(image[None])

2025-03-10 22:46:09.919353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-10 22:46:09.938621: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-10 22:46:09.944593: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 22:46:11.982061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ValueError: Size of label 'n' for operand 1 (196) does not match previous terms (12).

In [8]:
jax.tree.map(lambda x: x.shape, pred_outputs)

((1, 196, 768),
 {'encoded': (1, 196, 768),
  'encoder': ({'+mlp': (12, 1, 196, 768),
    '+sa': (12, 1, 196, 768),
    'mlp': (12, 1, 196, 768),
    'sa': (12, 1, 196, 768)},
   {('Dense', '__call__'): [(12, 1, 196, 768),
     (12, 1, 196, 768),
     (12, 1, 196, 768),
     (12, 1, 196, 768),
     (12, 1, 196, 3072),
     (12, 1, 196, 768)]}),
  'with_posemb': (1, 196, 768)})

In [9]:
callable(model.transformer.num_layers)

False

In [37]:
def cossim(a, b):
    return (a * b).sum() / np.sqrt((a * a).sum() * (b * b).sum() + 1e-8)

In [10]:
reference_outputs = test_equality._run_reference(params, image[None])

In [11]:
print(jax.tree.map(lambda x: x.shape, reference_outputs[1]))
# print(jax.tree.map(lambda x: x.shape, pred_outputs))

{'intermediates': {'Dropout_0': {'__call__': ((1, 196, 768),)}, 'MAPHead_0': {'LayerNorm_0': {'__call__': ((1, 1, 768),)}, 'MlpBlock_0': {'Dense_0': {'__call__': ((1, 1, 3072),)}, 'Dense_1': {'__call__': ((1, 1, 768),)}, 'Dropout_0': {'__call__': ((1, 1, 3072),)}, '__call__': ((1, 1, 768),)}, 'MultiHeadDotProductAttention_0': {'__call__': ((1, 1, 768),), 'key': {'__call__': ((1, 196, 12, 64),)}, 'out': {'__call__': ((1, 1, 768),)}, 'query': {'__call__': ((1, 1, 12, 64),)}, 'value': {'__call__': ((1, 196, 12, 64),)}}, '__call__': ((1, 768),)}, 'Transformer': {'__call__': (((1, 196, 768), {'block00': {'+mlp': (1, 196, 768), '+sa': (1, 196, 768), 'mlp': (1, 196, 768), 'sa': (1, 196, 768)}, 'block01': {'+mlp': (1, 196, 768), '+sa': (1, 196, 768), 'mlp': (1, 196, 768), 'sa': (1, 196, 768)}, 'block02': {'+mlp': (1, 196, 768), '+sa': (1, 196, 768), 'mlp': (1, 196, 768), 'sa': (1, 196, 768)}, 'block03': {'+mlp': (1, 196, 768), '+sa': (1, 196, 768), 'mlp': (1, 196, 768), 'sa': (1, 196, 768)}, '

In [48]:
jax.tree.map(lambda x: x.shape, params['Transformer']['encoderblock']['MultiHeadDotProductAttention_0']['value'])

{'bias': (12, 12, 64), 'kernel': (12, 768, 12, 64)}

In [54]:
model.transformer.layers.attn.value.kernel - params['Transformer']['encoderblock']['MultiHeadDotProductAttention_0']['value']['kernel'].reshape(12, 768, 768)

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [69]:
pred_mha[1].shape

(12, 1, 196, 768)

In [76]:
import flax.linen as nn
reference_mha = reference_outputs[1]['intermediates']['Transformer']['encoderblock']['MultiHeadDotProductAttention_0']
pred_mha =pred_outputs[1]['encoder'][1][('Dense', '__call__')]
a1 = nn.dot_product_attention(
    reference_mha['query']['__call__'][0][0],
    reference_mha['key']['__call__'][0][0],
    reference_mha['value']['__call__'][0][0],
)
a1.shape

(1, 196, 12, 64)

In [77]:
q = pred_mha[0][0].reshape(reference_mha['query']['__call__'][0][0].shape)
k = pred_mha[1][0].reshape(reference_mha['key']['__call__'][0][0].shape)
v = pred_mha[2][0].reshape(reference_mha['value']['__call__'][0][0].shape)
q = q / jnp.sqrt(q.shape[-1])
attn_logits = jnp.einsum("bqnh, bknh -> bnqk", q, k)
attn_weights = nn.softmax(attn_logits, axis=-1)
out = jnp.einsum("bnqk, bknh -> bqnh", attn_weights, v)

In [78]:
cossim(out, a1)

Array(1., dtype=float32)

In [74]:
cossim(attn_weights, a1)

Array(0.9999999, dtype=float32)

In [60]:
l1 = reference_outputs[1]['intermediates']['Transformer']['encoderblock']['MultiHeadDotProductAttention_0']['out']['__call__'][0][0]
l2 = pred_outputs[1]['encoder'][1][('Dense', '__call__')][3][0].reshape(l1.shape)
cossim(l1, l2)

Array(0.81712836, dtype=float32)

In [44]:
cossim(l1, l2)

Array(-0.01933866, dtype=float32)

In [13]:
dir(model.transformer.layers)

['__annotations__',
 '__call__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'attn',
 'create',
 'drop1',
 'drop2',
 'ln1',
 'ln2',
 'mlp']

In [ ]:
p = pred_outputs[1]['encoder']['+sa'][0]
r = reference_outputs[1]['encoder']['block00']['+sa']
cossim(p, r)

TypeError: tuple indices must be integers or slices, not str

In [21]:
def cossim(a, b):
    return (a * b).sum() / np.sqrt((a * a).sum() * (b * b).sum() + 1e-8)
print(cossim(reference_outputs[1]['with_posemb'], pred_outputs[1]['with_posemb']))
print(cossim(reference_outputs[1]['encoded'], pred_outputs[1]['encoded']))

1.0
0.4148036


In [12]:
from vit import image_embedding

In [13]:
embedding = image_embedding.ImageEmbedding.create_from_pretrained(params, 16)

In [14]:
import numpy as np
np.linalg.norm(
    embedding(image[None]) - reference_outputs[1]['with_posemb']
    )

2.866363e-05

In [15]:
import utils
expected_outputs = utils.load_pytree('vit/expected_output/')

/nfs/nfs2/users/dibya/miniconda3/envs/lv_env/lib/python3.11/site-packages/orbax/checkpoint/_src/serialization/type_handlers.py:1136: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Devic

ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERRO

ValueError: sharding passed to deserialization should be specified, concrete and an instance of `jax.sharding.Sharding`. Got None

In [7]:
jax.tree.map(lambda x: x.shape, params)

{'MAPHead_0': {'LayerNorm_0': {'bias': (768,), 'scale': (768,)},
  'MlpBlock_0': {'Dense_0': {'bias': (3072,), 'kernel': (768, 3072)},
   'Dense_1': {'bias': (768,), 'kernel': (3072, 768)}},
  'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 64),
    'kernel': (768, 12, 64)},
   'out': {'bias': (768,), 'kernel': (12, 64, 768)},
   'query': {'bias': (12, 64), 'kernel': (768, 12, 64)},
   'value': {'bias': (12, 64), 'kernel': (768, 12, 64)}},
  'probe': (1, 1, 768)},
 'Transformer': {'encoder_norm': {'bias': (768,), 'scale': (768,)},
  'encoderblock': {'LayerNorm_0': {'bias': (12, 768), 'scale': (12, 768)},
   'LayerNorm_1': {'bias': (12, 768), 'scale': (12, 768)},
   'MlpBlock_0': {'Dense_0': {'bias': (12, 3072), 'kernel': (12, 768, 3072)},
    'Dense_1': {'bias': (12, 768), 'kernel': (12, 3072, 768)}},
   'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 12, 64),
     'kernel': (12, 768, 12, 64)},
    'out': {'bias': (12, 768), 'kernel': (12, 12, 64, 768)},
    'query': {'

In [ ]:
import utils
utils.load_pytree('vit/expected_output/')

/nfs/nfs2/users/dibya/miniconda3/envs/lv_env/lib/python3.11/site-packages/orbax/checkpoint/_src/serialization/type_handlers.py:1136: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Device TPU_0(process=0,(0,0,0,0)) was not found in jax.local_devices().
ERROR:root:Devic

ValueError: sharding passed to deserialization should be specified, concrete and an instance of `jax.sharding.Sharding`. Got None

In [ ]:
checkpointer.metadata(path)[0].restore_type

AttributeError: 'ArrayMetadata' object has no attribute 'restore_type'

In [ ]:
d['embedding']

NameError: name 'd' is not defined